In [1]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\senyo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Now we load the json

In [2]:
document_path = r"../../data/fashion_products_dataset.json"

import sys
from pathlib import Path
project_root = Path().resolve().parent.parent
sys.path.append(str(project_root))
from myapp.search.load_corpus import load_corpus
docs = load_corpus(document_path)

In [3]:
display(docs["TKPFCZ9EA7H5FYZH"])

Document(pid='TKPFCZ9EA7H5FYZH', title='Solid Women Multicolor Track Pants', description='Yorker trackpants made from 100% rich combed cotton giving it a rich look.Designed for Comfort,Skin friendly fabric,itch-free waistband & great for all year round use Proudly made in India', brand='York', category='Clothing and Accessories', sub_category='Bottomwear', product_details={'Style Code': '1005COMBO2', 'Closure': 'Elastic', 'Pockets': 'Side Pockets', 'Fabric': 'Cotton Blend', 'Pattern': 'Solid', 'Color': 'Multicolor'}, seller='Shyam Enterprises', out_of_stock=False, selling_price=921.0, discount=69.0, actual_price=2999.0, average_rating=3.9, url='https://www.flipkart.com/yorker-solid-men-multicolor-track-pants/p/itmd2c76aadce459?pid=TKPFCZ9EA7H5FYZH&lid=LSTTKPFCZ9EA7H5FYZHVYXWP0&marketplace=FLIPKART&srno=b_1_1&otracker=browse&fm=organic&iid=177a46eb-d053-4732-b3de-fcad6ff59cbd.TKPFCZ9EA7H5FYZH.SEARCH&ssid=utkd4t3gb40000001612415717799', images=['https://rukminim1.flixcart.com/image/128/1

In [6]:
display(docs["TKPFCZ9EA7H5FYZH"].title)

'Solid Women Multicolor Track Pants'

In [4]:
print("Total documents:", len(docs))

Total documents: 28080
